<a href="https://colab.research.google.com/github/Maruf-S/WordEmbeddingsWithLTSM/blob/main/WordPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from tensorflow.keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np
import nltk
nltk.download('punkt')

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#Load the text files in a plain text format to train the mdel
from google.colab import files
path_to_file = list(files.upload().keys())[0]

Saving Doc1.txt to Doc1 (1).txt


In [ ]:
#Vocab
# example = text.replace("\n", " ")
import string
##
example = text
# print(example[:100])
##

# from nltk.tokenize import word_tokenize
# example = word_tokenize(example)
print(example[:15])
# example = example.lower()
# example = example.split()
# print(example[:100])
# import string
table = str.maketrans('', '', string.punctuation)
example = [w.translate(table) for w in example]
vocab = sorted(set(example))
print(len(vocab))
print(vocab[:100])
voc_size = len(vocab)
len(text)


	

badlands
43
['', '\t', '\n', '\r', ' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '“', '”']


1077841

In [ ]:
# def text_to_int(text):
#   return np.array([char2idx[c] for c in text])

# text_as_int = text_to_int(text)

In [ ]:
# Creating a mapping from words characters to ints
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
def word_to_int(text):
  return np.array([char2idx[c] for c in example])

word_as_int = word_to_int(text)
# print(type(list(word_as_int[:10])))
print("////////////////////////")
# print(word_to_int(""))

////////////////////////


In [ ]:
# onehot_repr=[one_hot(words,voc_size)for words in example] 
# arr = np.array(onehot_repr)
# print(arr)
# # onehot_repr = [ int(x) for x in onehot_repr ]#get rid of the sublist
# # print(onehot_repr)

In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(word_as_int[:13]))

badlandsbyterencemalickfinalversiondialogueandcontinuityfadeinint.


In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(VOCAB_SIZE, 32),
#     tf.keras.layers.LSTM(32),
#     tf.keras.layers.Dense(1, activation="sigmoid")
# ])

In [ ]:
seq_length = 10 # length of sequence for a training example
examples_per_epoch = len(example)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(word_as_int)

Next we can use the batch method to turn this stream of characters into batches of desired length.

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
badlandsbyterencemalickfinalversiondialogueandcontinuityfade

OUTPUT
byterencemalickfinalversiondialogueandcontinuityfadein


EXAMPLE

INPUT
int.bedroomthetimeis1959.hollysargis,

OUTPUT
.bedroomthetimeis1959.hollysargis,the


In [ ]:
BATCH_SIZE = 64
voc_size = voc_size  # vocab is number of unique words
EMBEDDING_DIM = 30
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model
model = build_model(voc_size,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 30)            312750    
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          4321280   
_________________________________________________________________
dense_1 (Dense)              (64, None, 10425)         10685625  
Total params: 15,319,655
Trainable params: 15,319,655
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 10, 10425) # (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-3.95137176e-05  9.44193627e-04 -1.74029003e-04 ... -7.21865566e-04
    5.24793686e-05  4.57966089e-04]
  [ 3.60347214e-04  8.56439292e-05  3.78956756e-04 ... -3.62775434e-04
   -3.26882000e-04  4.33132955e-04]
  [ 7.44409626e-04 -2.20457659e-04 -2.40832567e-04 ...  3.50860966e-04
   -3.60447943e-04  2.82003777e-04]
  ...
  [ 4.64327559e-06  3.80402314e-04  1.90953258e-04 ... -5.66780451e-04
    1.24066602e-04  1.03855855e-03]
  [ 9.92214609e-06 -4.16419614e-04  8.02315539e-04 ...  1.22168685e-05
    7.77290377e-04  6.39292470e-04]
  [ 4.44218138e-04  6.41159859e-05  1.05696975e-03 ...  4.45553560e-05
    8.14540486e-04  9.24728636e-04]]

 [[-4.60212352e-04  5.70160395e-04 -3.79910925e-04 ... -2.45814357e-04
   -3.39056656e-04 -7.61182018e-05]
  [-4.40537522e-04  5.21148031e-04 -6.25294750e-04 ... -2.27550714e-04
    1.71605061e-05 -4.03533544e-04]
  [-3.44105822e-04  1.15747878e-03 -7.31018954e-04 ... -8.32607679e-04
    5.89771844e-05  2.47059477e-04]
  ...
  [-3.610

In [ ]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

10
tf.Tensor(
[[-3.95137176e-05  9.44193627e-04 -1.74029003e-04 ... -7.21865566e-04
   5.24793686e-05  4.57966089e-04]
 [ 3.60347214e-04  8.56439292e-05  3.78956756e-04 ... -3.62775434e-04
  -3.26882000e-04  4.33132955e-04]
 [ 7.44409626e-04 -2.20457659e-04 -2.40832567e-04 ...  3.50860966e-04
  -3.60447943e-04  2.82003777e-04]
 ...
 [ 4.64327559e-06  3.80402314e-04  1.90953258e-04 ... -5.66780451e-04
   1.24066602e-04  1.03855855e-03]
 [ 9.92214609e-06 -4.16419614e-04  8.02315539e-04 ...  1.22168685e-05
   7.77290377e-04  6.39292470e-04]
 [ 4.44218138e-04  6.41159859e-05  1.05696975e-03 ...  4.45553560e-05
   8.14540486e-04  9.24728636e-04]], shape=(10, 10425), dtype=float32)


In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

10425
tf.Tensor(
[-3.9513718e-05  9.4419363e-04 -1.7402900e-04 ... -7.2186557e-04
  5.2479369e-05  4.5796609e-04], shape=(10425,), dtype=float32)


In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'tillwonderprojectsmammockeddictatorhatchceremoniousincurministerprosecute'

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(data, epochs=100, callbacks=[checkpoint_callback])
model.summary()

In [ ]:
model = build_model(voc_size, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 30)             312750    
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 1024)           4321280   
_________________________________________________________________
dense_2 (Dense)              (1, None, 10425)          10685625  
Total params: 15,319,655
Trainable params: 15,319,655
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of words to generate
  num_generate = 30

  # Converting our start string to numbers (vectorizing)
  # input_eval = [char2idx[s] for s in start_string]
  input_eval = [char2idx[s] for s in start_string.split()] 
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + " " + ' '.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: color
color santa explains them , gentle cominius : do not hear , we 'll muster in my feet . but tell me not me to say . guess you 're on
